In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from scipy.stats import norm, skew #for some statistics
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, LinearSVR
from sklearn.neural_network import MLPRegressor
# something about warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
#pd.options.mode.chained_assignment = None

#Machine learning libraries
#import keras (This is only commented out because Aviv couldn't install keras on his conda.  He will fix this eventually)
import sklearn
from sklearn.preprocessing import LabelEncoder

# changes display options to allow many rows and columns to be displayed
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
# uploads data sets
train = pd.read_csv("../Data/House Prices Advanced Regression Techniques/train.csv")
test = pd.read_csv("../Data/House Prices Advanced Regression Techniques/test.csv")
train=train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)
total = pd.concat([train, test], axis=0)
total2 =total.copy()
# changes display options to allow many rows and columns to be displayed
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
# makes an indicator variable for whether there was LotFrontage was NA
cond = total2['LotFrontage'].isnull()
total2['HasLotFrontage'] = cond * 1

# gets LotFrontage from sqrt(LotArea) for a new LotFrontage variable
total2['LotFrontage2'] = total2['LotFrontage']
sqrtlotarea = np.sqrt(total2['LotArea'])
total2['LotFrontage2'][cond] = sqrtlotarea[cond]

# changes NaN's in GarageType to "None" and imputes yearbuilt for garageyrblt for houses with no garages
cond = total2['GarageType'].isnull()
total2['GarageType'][cond] = "None"
total2['GarageYrBlt'][cond] = total2['YearBuilt'][cond]

# imputes -1 for missing values in MasVnrArea
total2.MasVnrArea = total2.MasVnrArea.fillna(-1)

# removes changes null values that shouldn't be treated as null
total2['Alley'] = total2['Alley'].fillna("None")
total2['BsmtQual'] = total2['BsmtQual'].fillna("None")
total2['BsmtCond'] = total2['BsmtCond'].fillna("None")
total2['BsmtExposure'] = total2['BsmtExposure'].fillna("None")
total2['BsmtFinType1'] = total2['BsmtFinType1'].fillna("None")
total2['BsmtFinType2'] = total2['BsmtFinType2'].fillna("None")
total2['FireplaceQu'] = total2['FireplaceQu'].fillna("None")
total2['GarageType'] = total2['GarageType'].fillna("None")
total2['GarageFinish'] = total2['GarageFinish'].fillna("None")
total2['GarageQual'] = total2['GarageQual'].fillna("None")
total2['GarageCond'] = total2['GarageCond'].fillna("None")
total2['PoolQC'] = total2['PoolQC'].fillna("None")
total2['Fence'] = total2['Fence'].fillna("None")
total2['MiscFeature'] = total2['MiscFeature'].fillna("None")
total2['MasVnrType'] = total2['MasVnrType'].fillna("missing")

# recodes some categorical variables as ordinal
total2 = total2.replace({"Alley" : {"None" : 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"None" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageFinish" : {"None" : 0, "Unf" : 1, "RFn" : 2, "Fin" : 3},
                       "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}})

# fixing specific na cells
total2.BsmtFinSF1 = total2.BsmtFinSF1.fillna(0)
total2.BsmtFinSF2 = total2.BsmtFinSF2.fillna(0)
total2.BsmtFullBath = total2.BsmtFullBath.fillna(0)
total2.BsmtHalfBath = total2.BsmtHalfBath.fillna(0)
total2.BsmtUnfSF = total2.BsmtUnfSF.fillna(0)


total2 = total2.fillna(total2.mean())
total2 = total2.apply(lambda x:x.fillna(x.value_counts().index[0]))
total2.SalePrice = np.log(total2.SalePrice) ##log transform for the response var
total2['MSSubClass']=total2['MSSubClass'].apply(str)
total2['YrSold'] = total2['YrSold'].astype(str)
total2['MoSold'] = total2['MoSold'].astype(str)
total2['TotalSF'] = total2['TotalBsmtSF'] + total2['1stFlrSF'] + total2['2ndFlrSF']

intFeature = total2.select_dtypes(include=["int64"]).apply(lambda x: skew(x.astype(float).dropna()))
skewIntFeat = intFeature[intFeature > .75].index.tolist()
total2[skewIntFeat] = total2[skewIntFeat].apply(np.log1p)
total2.drop("Id", axis=1, inplace=True)
total2.drop("LotFrontage",axis=1,inplace=True)
#total2.drop("HasLotFrontage",axis=1,inplace=True)

catCol = total2.select_dtypes(["object"]).columns.tolist()
LabelMap = dict()
for i in catCol:
    print('Start encoding ' + i)
    total2[i] = total2[i].fillna('missing')
    le = LabelEncoder()
    le.fit(total2[i].astype(str))
    LabelMap[i] = dict(zip(le.classes_, le.transform(le.classes_)))
#total2.replace(LabelMap, inplace=True)
total2 = pd.get_dummies(total2)
##

Start encoding BldgType
Start encoding BsmtExposure
Start encoding CentralAir
Start encoding Condition1
Start encoding Condition2
Start encoding Electrical
Start encoding Exterior1st
Start encoding Exterior2nd
Start encoding Fence
Start encoding Foundation
Start encoding GarageType
Start encoding Heating
Start encoding HouseStyle
Start encoding LandContour
Start encoding LotConfig
Start encoding MSSubClass
Start encoding MSZoning
Start encoding MasVnrType
Start encoding MiscFeature
Start encoding MoSold
Start encoding Neighborhood
Start encoding RoofMatl
Start encoding RoofStyle
Start encoding SaleCondition
Start encoding SaleType
Start encoding YrSold


In [3]:
total2.shape

(2917, 263)

In [4]:
dtrain = total2[:train.shape[0]]
X_train = dtrain.drop("SalePrice", axis=1)
y_train = dtrain['SalePrice']
dtest = total2[train.shape[0]:]
X_test = dtest.drop("SalePrice", axis=1)
y_test = dtest['SalePrice']

In [5]:
## LightGBM
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
model_lgb.fit(X_train, y_train)
lgb_train_pred = model_lgb.predict(X_train)
lgb_pred = np.expm1(model_lgb.predict(X_test))
print(rmsle(y_train, lgb_train_pred))

0.0729484397415


In [6]:
## XGBoost
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb.fit(X_train, y_train)
xgb_train_pred = model_xgb.predict(X_train)
xgb_pred = np.expm1(model_xgb.predict(X_test))
print(rmsle(y_train, xgb_train_pred))

0.0795760656427


In [7]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005))
#Lasso = Lasso(alpha=0.0005,max_iter=10000)
#ridge = Ridge(alpha=60)
Lasso = make_pipeline(RobustScaler(), Lasso(alpha = np.exp(-8), normalize=False ))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
#SVRr = SVR(C= 13, epsilon= 0.009, gamma= 0.0004,kernel='rbf' )
SVRr = SVR(C = 0.2, epsilon = 0.1, kernel = 'linear' )
BR = BayesianRidge()
NN = make_pipeline(
    RobustScaler(),
    MLPRegressor(
                hidden_layer_sizes = (200, 20),
                activation = "logistic",
                solver = 'lbfgs',
                alpha = 0.001,
                random_state = 0
                )
)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber')

In [9]:
# Stacking
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR,SVRr,BR,NN),
                                                 meta_model = lasso)

#print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
%%time
stacked_averaged_models.fit(np.array(X_train), np.array(y_train))
stacked_train_pred = stacked_averaged_models.predict(X_train)
stacked_pred = np.expm1(stacked_averaged_models.predict(X_test))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [ ]:
subcsv = pd.DataFrame({'Id': test['Id'], 'SalePrice': ensemble})
subcsv.to_csv("prediction1nopc.csv", index=False)
subcsv.head()